In [1]:
import pandas as pd
import skmob
from skmob.preprocessing import filtering
from warnings import filterwarnings
filterwarnings('ignore')

In [14]:
formatted_sample_df = pd.read_csv('./train_formatted_sample.csv')

In [15]:
# Transforma os dados para o objeto TrajDataFrame
sample_tdf = skmob.TrajDataFrame(formatted_sample_df, latitude='lat', longitude='lon', user_id='taxi_id', datetime='timestamp', crs={'init': 'WGS84'})

In [16]:
def plot_trajectory_diff(tdf: skmob.TrajDataFrame, tdf2: skmob.TrajDataFrame, zoom = 15):
  diff_points = tdf[~tdf.apply(tuple,1).isin(tdf2.apply(tuple,1))]
  map_f = diff_points.plot_stops(hex_color='green', zoom = zoom)
  map_f = tdf.plot_trajectory(map_f=map_f, dashArray='5,5', start_end_markers=False, hex_color='red')
  map_f = tdf2.plot_trajectory(map_f=map_f, start_end_markers=False, hex_color='blue')
  return map_f

In [17]:
# Pega a a trajetória com id '1373395476620000246', como exemplo
one_tdf = sample_tdf[sample_tdf['trip_id'] == 1373395476620000246]
# Computa uma trajetória filtrada, usando o critério de velocidade máxima de 300km/h
one_tdf_filtered = filtering.filter(one_tdf, max_speed_kmh=160)
print('Número de pontos de ruído encontrados', len(one_tdf) - len(one_tdf_filtered))

Número de pontos de ruído encontrados 2


In [18]:
plot_trajectory_diff(one_tdf, one_tdf_filtered, zoom=14)

In [19]:
# Aplica o filtro para toda a amostra
sample_filtered = filtering.filter(sample_tdf, max_speed_kmh=160)

In [20]:
sample_filtered.to_csv('./train_filtered_sample.csv', index=False)